In [1]:
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import *

In [20]:
spark = (SparkSession
        .builder
        .appName("Spark SQL Example 1")
        .getOrCreate())

23/11/16 10:17:22 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
csv_file = "flights/departuredelays.csv"

In [5]:
df = (spark.read.format("csv")
      .option("inferschema", "true")
      .option("header", "true")
      .load(csv_file))

In [6]:
df.createOrReplaceTempView("us_delay_flights_tbl")

Now we can issue standard SQL queries. These would be no differenet than what we would
do in a standrad relational database.

The data has five columns:
* date
* delay (in minutes). Negative numbers show early departures.
* distance from origin airport to destimation airport
* origin -- airport code
* destination -- airport code

In [10]:
spark.sql("""SELECT distance, origin, destination
FROM us_delay_flights_tbl WHERE distance > 1000
ORDER BY distance DESC""").show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [11]:
spark.sql("""SELECT date, delay, origin, destination
FROM us_delay_flights_tbl 
WHERE delay > 120 and ORIGIN = 'SFO' and DESTINATION = 'ORD'
ORDER by delay DESC""").show(10)

+-------+-----+------+-----------+
|   date|delay|origin|destination|
+-------+-----+------+-----------+
|2190925| 1638|   SFO|        ORD|
|1031755|  396|   SFO|        ORD|
|1022330|  326|   SFO|        ORD|
|1051205|  320|   SFO|        ORD|
|1190925|  297|   SFO|        ORD|
|2171115|  296|   SFO|        ORD|
|1071040|  279|   SFO|        ORD|
|1051550|  274|   SFO|        ORD|
|3120730|  266|   SFO|        ORD|
|1261104|  258|   SFO|        ORD|
+-------+-----+------+-----------+
only showing top 10 rows



As an exercise, convert the date column into a readable format and find the months where the
delays were most common. Below we show a more complex query.

In [14]:
spark.sql("""SELECT delay, origin, destination,
            CASE
                WHEN delay > 360 THEN 'Very Long Delays'
                WHEN delay >= 120 and delay <= 360 THEN 'Long Delays'
                WHEN delay >= 60 and delay < 120 THEN 'Short Delays'
                WHEN delay > 0 and delay < 60 THEN 'Tolerable Delays'
                WHEN delay == 0 THEN 'No Delays'
                ELSE 'Early'
            END AS Flight_Delays
            FROM us_delay_flights_tbl
            ORDER BY origin, delay DESC""").show(10)



+-----+------+-----------+-------------+
|delay|origin|destination|Flight_Delays|
+-----+------+-----------+-------------+
|  333|   ABE|        ATL|  Long Delays|
|  305|   ABE|        ATL|  Long Delays|
|  275|   ABE|        ATL|  Long Delays|
|  257|   ABE|        ATL|  Long Delays|
|  247|   ABE|        ATL|  Long Delays|
|  247|   ABE|        DTW|  Long Delays|
|  219|   ABE|        ORD|  Long Delays|
|  211|   ABE|        ATL|  Long Delays|
|  197|   ABE|        DTW|  Long Delays|
|  192|   ABE|        ORD|  Long Delays|
+-----+------+-----------+-------------+
only showing top 10 rows



As an exercise, convert these SQL queries to DataFrame operations. One example is given below.

In [19]:
(df.select("distance", "origin", "destination")
 .where(col("distance") > 1000)
 .orderBy(desc("distance"))).show(10)

+--------+------+-----------+
|distance|origin|destination|
+--------+------+-----------+
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
|    4330|   HNL|        JFK|
+--------+------+-----------+
only showing top 10 rows



In [21]:
spark.stop()